## Documentation

To read more about the bulk API, visit the [docs](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html).



## Connect to ElasticSearch

In [ ]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

## Without bulk API

In [ ]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

Let's index the documents sequentially.

In [ ]:
import json
from tqdm import tqdm


document_ids = []
dummy_data = json.load(open("../data/dummy_data.json"))

for document in tqdm(dummy_data, total=len(dummy_data)):
    response = es.index(index='my_index', body=document)
    document_ids.append(response['_id'])

Let's update the first and second documents

In [ ]:
from pprint import pprint

response = es.update(
    index="my_index",
    id=document_ids[0],
    script={
        "source": "ctx._source.title = params.title",
        "params": {
            "title": "New Title"
        }
    },
)
pprint(response.body)

In [ ]:
response = es.update(
    index="my_index",
    id=document_ids[1],
    script={
        "source": "ctx._source.new_field = 'dummy_value'",
    },
)
pprint(response.body)

Let's delete the third document

In [ ]:
response = es.delete(index="my_index", id=document_ids[2])
pprint(response.body)

We executed each operation one at a time, with each action requiring a separate API call. This approach is slow and inefficient. Now, let’s see how to accomplish the same task using the bulk API.

## Bulk API

In [ ]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

In [ ]:
response = es.bulk(
    operations=[
        # Action 1
        {
            "index": {
                "_index": "my_index",
                "_id": "1"
            }
        },
        # Source 1
        {
            "title": "Sample Title 1",
            "text": "This is the first sample document text.",
            "created_on": "2024-09-22"
        },
        # Action 2
        {
            "index": {
                "_index": "my_index",
                "_id": "2"
            }
        },
        # Source 2
        {
            "title": "Sample Title 2",
            "text": "Here is another example of a document.",
            "created_on": "2024-09-24"
        },
        # Action 3
        {
            "index": {
                "_index": "my_index",
                "_id": "3"
            }
        },
        # Source 3
        {
            "title": "Sample Title 3",
            "text": "The content of the third document goes here.",
            "created_on": "2024-09-24"
        },
        # Action 4
        {
            "update": {
                "_id": "1",
                "_index": "my_index"
            }
        },
        # Source 4
        {
            "doc": {
                "title": "New Title"
            }
        },
        # Action 5
        {
            "update": {
                "_id": "2",
                "_index": "my_index"
            }
        },
        # Source 5
        {
            "doc": {
                "new_field": "dummy_value"
            }
        },
        # Action 6
        {
            "delete": {
                "_index": "my_index",
                "_id": "3"
            }
        },
    ],
)

pprint(response.body)

If `errors` is `False`, it means the bulk API successfully executed all the actions.

In [ ]:
response.body["errors"]